In [2]:
import numpy as np
from skimage import feature, filters
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle
import cv2
from skimage.feature import hog
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
def extract_features(images):
    features = []
    
    for image in images:
        # Resize the image to at least 16x16 pixels
        resized_image = cv2.resize(image, (16, 16))

        # Normalize the pixel values to 0-1
        normalized_image = resized_image / 255.0

        # Compute the HoG features
        hog_features = hog(normalized_image, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), visualize=False, feature_vector=True)
        
        # Perform Canny Edge Detection on the normalized image
        edges = cv2.Canny((normalized_image * 255).astype('uint8'), 100, 200)

        # Flatten the edges image to a 1D array
        edges_flat = edges.flatten()

        # Combine the HoG features and Canny edges into a single feature vector
        combined_features = np.concatenate((hog_features, edges_flat))

        features.append(combined_features)
    
    return features

In [4]:
def train_model(X, Y, epochs=10):
    # Extract features from training data
    x_train_features = extract_features(X)

    # Set up classifiers
    knn = KNeighborsClassifier()
    rf = RandomForestClassifier()

    # Set up voting classifier
    voting_classifier = VotingClassifier(estimators=[('knn', knn), ('rf', rf)], voting='hard')

    # Set up parameter grid for hyperparameter tuning
    param_grid = {'knn__n_neighbors': [3, 5, 7], 'rf__n_estimators': [50, 100, 200]}

    # Perform grid search cross-validation
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=5)
    grid_search.fit(x_train_features, Y)

    # Get best estimator and best parameters
    best_classifier = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Train final model with best parameters and specified number of epochs
    for _ in range(epochs):
        best_classifier.fit(x_train_features, Y)

    return best_classifier, best_params

In [5]:
if __name__ == '__main__':
    # Load training data
    Xtr = np.loadtxt("/content/drive/MyDrive/TrainData.csv")
    Ytr = np.loadtxt("/content/drive/MyDrive/TrainLabels.csv")

    # Reshape training data to 2D array
    Xtr = Xtr.reshape((Xtr.shape[0], -1))

    # Train the model
    trained_model, best_parameters = train_model(Xtr, Ytr)

    # Save the final model
    pickle.dump(trained_model, open("myModel.pkl", "wb"))